## Introduction
### This nootbook was created to study and understand the following great notebook in my own way.  
[Cassava / resnext50_32x4d starter](https://www.kaggle.com/yasufuminakama/cassava-resnext50-32x4d-starter-training)  
### I am deeply grateful to Y.Nakama!!

This may only be for my own reference...

In [ ]:
import os

os.listdir("../input")

In [ ]:
#
import os
import random
import re

import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import cv2

import matplotlib.pyplot as plt
import seaborn as sns

#sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

#Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import(
    CosineAnnealingWarmRestarts,CosineAnnealingLR,
    ReduceLROnPlateau
    )

from albumentations import(
    Compose,OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop,
    HorizontalFlip, VerticalFlip, RandomBrightness, RandomContrast,
    RandomBrightnessContrast, Rotate, ShiftScaleRotate, Cutout,
    IAAAdditiveGaussianNoise, Transpose
    )
from albumentations.pytorch import ToTensor, ToTensorV2
from albumentations import ImageOnlyTransform

import time
import datetime

import warnings
warnings.filterwarnings('ignore')
#

#device = torch.device('cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#-------------------------------------------
# Path and Data Loading
#-------------------------------------------

data_top = "../input/"
commpe_name = "cassava-leaf-disease-classification"
data_path = data_top + commpe_name

OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

#labels
label_map = pd.read_json(os.path.join(data_path, 
                                      "label_num_to_disease_map.json"),
                         orient = "index")
label_names = label_map.iloc[:,0].values

#表記の短縮
new_names = []
for name in [val for val in label_names]:
    name = name.replace(" ", "")
    name = re.sub("^Cassava", "", name)
    name = re.sub("\(.+\)$", "", name)
    new_names.append(name)

new_map = dict(zip(np.arange(len(new_names)), new_names))

#train
train = pd.read_csv(os.path.join(data_path, "train.csv"))
train.columns
train['label'].value_counts().reset_index().sort_values("index")

train['class_name'] = train['label'].map(new_map)

#test
test = pd.read_csv(os.path.join(data_path, "sample_submission.csv"))

### Creating a confirmation dataset

Create testing dataset to see how the augmentaion works.

In [ ]:
#---------------------------------------------
# Dataset for Observation
#---------------------------------------------

N_DATA = 4 # サンプル数はこの5倍
DEBUG = False # 画像データ構造確認用

random.seed(11)

#label順にサンプルを1個づつ取り出す
tmp = []
df_sample = pd.DataFrame()
for i in range(N_DATA):
    #print(i)
    tmp = [train[train['label'] == val].sample(1) for val in range(5)]
    df_tmp = pd.concat(tmp, axis = 0)
    df_sample = pd.concat([df_sample, df_tmp], axis = 0)

# Normarizeをかけずにaugmenttion画像を確認するのに使用する。

### Class for dataset

In [ ]:
#--------------------------------------------
# Dataset
#--------------------------------------------

class SampleDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df
        self.file_names = df['image_id'].values
        self.labels = df['label'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
        
    def __getitem__(self, index):
        file_name = self.file_names[index]
        file_path = os.path.join(data_path, "train_images", file_name)
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if DEBUG:
            print("shape = ", image.shape)
            print(type(image))
            im = image[:, :, 0]
            print(np.max(im), np.min(im))
            plt.imshow(image)
            plt.title("Original")
            plt.show()
            
        trans_org  = Compose([Resize(255,255), ToTensorV2()])
        image_org = trans_org(image = image)
        image_org = image_org['image']
        
        if DEBUG:
            print(image_org.shape)

        if self.transform:
            augmented = self.transform(image = image)
            image_aug = augmented['image']
            if DEBUG:
                print("After trans shpe = ", image_aug.shape)
                image_np = image_aug.cpu().numpy()#image
                image_np = np.rollaxis(image_np, 0, 3)#image
                print(image_np.shape)
                im = image_np[:, :, 0]
                print(np.max(im), np.min(im))
                   
        image = image_aug
        
        label = torch.tensor(self.labels[index]).long()
        
        if DEBUG: print("Return Shape = ", image.shape)
        
        return image, image_org, label
        # images after augmentation, original images, labels

In [ ]:
def get_transforms(aug_list, data):
    
    if data == "train": 
        return Compose(aug_list)
    
    elif data == 'valid':
        return Compose([Resize(255, 255),
                        Normalize(mean=[0.485, 0.456, 0.406], 
                                  std=[0.229, 0.224, 0.225]),
                        ToTensorV2()
                        ])

    return

#augmentationの内容はリストで受け渡しすることにした

**Function for plot**

In [ ]:
def image_for_plt(sample_loader):

    image_data = torch.empty(0,0,0,0)
    for (image, image_org, label) in sample_loader:
        #print(image.shape)

        if JOIN:
            image = torch.cat([image_org, image], dim = 0)
            if DEBUG: print("image shape = ", image.shape)

        if DEBUG:
            image_np = image.cpu().numpy()
            label_np = label.cpu().numpy()
            image_np.shape
            image_np2 = np.rollaxis(image_np, 1, 4)
            plt.imshow(image_np2[1])
            np.max(image_np2[0])
            np.min(image_np2[0])
            image.shape

        if len(image_data) == 0:
            image_data = image
        else:
            image_data = torch.cat([image_data, image], dim = 0)
            
    return image_data

# 表示用画像データセット作成。
# JOIN=1のときはオリジナルとaugmentation 画像を上下に並べて表示。
#　DataLoaderで読み出すと、画像のチャンネルが最初にくるので、np.rollaxis で入れ替える。

### Preparation to select a combination of argumentaion

In [ ]:
#-----------------------------------
# DataFrame to define Augmentaion
#-----------------------------------

df_aug = pd.DataFrame()
df_aug['item'] = ["RRC", "TR", "HF", "VF", "SSR", "NOR","TTR"]

df_aug["augment"] = [
    RandomResizedCrop(255, 255),
    Transpose(p = 0.5),
    HorizontalFlip(p = 0.5),
    VerticalFlip(p = 0.5),
    ShiftScaleRotate(p=1),
    Normalize(mean=[0.485, 0.456, 0.406], 
              std=[0.229, 0.224, 0.225]),
    ToTensorV2()
    ]

# RandomResizedCropは、サイズを揃えるために必ず入れる。
# もっとスマートなやり方がありそうだが。。。

### Let's confirm the images after augmentation

In [ ]:
random.seed(11)
DEBUG = 0
JOIN = 1 # オリジナルデータとくっつけるとき使用

aug_list = df_aug.query('\
                        item == "RRC" |\
                        item == "TR" |\
                        item == "HF" |\
                        item == "VF" |\
                        item == "SSR" |\
                        item == "" |\
                        item == "TTR"')['augment'].values

# やめたい変換、例えば　"TR" を　””とする。上記は"NOR"をやめる。

sample_data = SampleDataset(df_sample, 
                            transform = get_transforms(aug_list, data = "train"))

sample_loader = DataLoader(sample_data, 
                           batch_size = 5,
                           shuffle=False)

In [ ]:
image_data = image_for_plt(sample_loader)

In [ ]:
#----------------------------------------
# Vilualization
#----------------------------------------

image_data_np = image_data.cpu().numpy()
image_data_np.shape
image_data_np = np.rollaxis(image_data_np, 1, 4)

fig = plt.figure(figsize=(12, 24))
#fig.suptitle("The upper row is original (with resizing), the lower row is Augmented\n without normalizing")

for i in range(N_DATA):
    for j in range(10):
        image_number = i * 10 + j
        image_show = image_data_np[image_number]
        fig.add_subplot(8, 5, image_number + 1)
        plt.imshow(image_show)
        try:
            title_show = new_names[image_number - i*10]
            plt.title(title_show)
        except:
            if JOIN:
                plt.title('Augmented')
            else:
                pass

### Confirm normalized data images

In [ ]:
DEBUG = 0
JOIN = 0 # オリジナルデータとくっつけるとき使用

aug_list = df_aug.query('\
                        item == "RRC" |\
                        item == "TR" |\
                        item == "HF" |\
                        item == "VF" |\
                        item == "SSR" |\
                        item == "NOR" |\
                        item == "TTR"')['augment'].values

sample_data = SampleDataset(df_sample, 
                            transform = get_transforms(aug_list, data = "train"))

sample_loader = DataLoader(sample_data, 
                           batch_size = 5,
                           shuffle=False)

# パラメータは以下のため、Normalizeといっても、平均≒0.45、std≒0.22としている。
# Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# 平均=0, std = 1　ではない。

In [ ]:
image_data = image_for_plt(sample_loader)

### Images applied normalization

In [ ]:
#----------------------------------------
# Vilualization of the normalized data
#----------------------------------------

image_data_np = image_data.cpu().numpy()
image_data_np.shape
image_data_np = np.rollaxis(image_data_np, 1, 4)

fig = plt.figure(figsize=(12, 10))
for i in range(20):
    image_number = i 
    image_show = image_data_np[image_number]
    fig.add_subplot(4, 5, image_number + 1)
    plt.imshow(image_show)
    fig.suptitle('Augmented images with normalization')

## Next, let's build a CV with a simple CNN.

In [ ]:
#!pip -q install torchsummary

# インターネットがONできないので、使用できない。
# inference と分ければ使用できる。

### Chaeck train labels

In [ ]:
print(train['label'].value_counts())
print(train['label'].value_counts()[3]/train.shape[0])

Due to imbalanced data, the correct answer rate is 61.5% even if all are predicted to be level 3.  
I think that the correct answer rate needs to be 62% or more.

### Create a simple network with four convolution layers.

In [ ]:
#from torchsummary import summary
# 上記でモデルの確認ができる。

#-----------------------------
#  Moddel
#-----------------------------

class CnnTrial(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.conv3 = nn.Conv2d(32, 64, 5)
        self.conv4 = nn.Conv2d(64, 128, 5)
        
        self.bn1 = nn.BatchNorm2d(16)
        self.bn2 = nn.BatchNorm2d(32)
        self.bn3 = nn.BatchNorm2d(64)
        self.bn4 = nn.BatchNorm2d(128)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.drop = nn.Dropout2d(p = 0.5)
        
        self.relu = nn.ReLU(inplace = True)
        self.fc1 = nn.Linear(128*12*12, 1024)
        self.fc2 = nn.Linear(1024, 5)
        
        self.softmax = nn.Softmax(dim = 1)
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.bn1(x) #batch_norm がないと収束しない。
        x = self.relu(x)
        x = self.pool(x)
        #x = self.drop(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.pool(x)
        
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.pool(x)
    
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        #x = self.softmax(x) # 不要、ロス計算時に考慮されている
        
        return x
        
model = CnnTrial() 
print(model) 

#summary(model.to('cuda'),(3,255,255)) 
# モデルの構造、OK/NGの確認ができる。

### Then, let's start classifications!!

In [ ]:
#-------------------------------------------------
# Configration and helper functions
#-------------------------------------------------

class CFG:
    debug = False
    epochs = 12
    n_fold = 5
    num_workers = 4
    seed = 11


class AverageMeter(object):
    
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        
    def update(self, val, n = 1):
        self.val = val
        self.sum += val *n
        self.count += n
        self.avg = self.sum / self.count
#

def get_score(y_true, y_pred):
    return accuracy_score(y_true, y_pred)


def get_result(result_df):
    preds = result_df['preds'].values
    labels = result_df['label'].values
    score = get_score(labels, preds)
    #print(f'oof_score = {score:<.5f}')
    return score


def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

In [ ]:
#===== training =====

def train_fn(train_loader, model, criterion, optimizer, 
             epoch, scheduler, device):
       
    model.train()
    
    loss_step = []
    losses = AverageMeter()
    for step, (images, _, labels) in enumerate(train_loader):
        
        images = images.to(device)
        labels = labels.to(device)
        #上記がないとCPU/GPU不整合エラーとなる
        #!!!!! images.to(device)だとNGとなる　!!!
        
        batch_size = labels.size(0)
        
        y_pred = model(images)
       
        loss = criterion(y_pred, labels)
        if CFG.debug: print("loss = ", loss.item())
        loss_step.append(loss)
        losses.update(loss.item(), batch_size)
    
        loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(),1000)
        if CFG.debug: print("grad = ", grad_norm)
        optimizer.step()
        optimizer.zero_grad()
        
    if CFG.debug: plt.plot(loss_step[50:])

    print(f'Training Results\nloss_avg = {losses.avg:.3f}\ngrad = {grad_norm:.3f}')
    
    return losses.avg

#===== validation =====

def valid_fn(valid_loader, model, criterion, device):

    losses = AverageMeter()

    model.eval()
    
    preds = []
    for step, (images, _, labels) in enumerate(valid_loader):
        
        images = images.to(device)
        labels = labels.to(device)
        #上記がないとCPU/GPU不整合エラーとなる
        #!!!!! images.to(device)だとNGとなる　!!!
        
        batch_size = labels.size(0)
        
        with torch.no_grad():
            y_pred = model(images)
        loss = criterion(y_pred, labels)
        losses.update(loss.item(), batch_size)
        preds.append(y_pred.softmax(1).to('cpu').numpy())
        
        if CFG.debug: print(f'Loss = {loss}')
        #if step%50 ==0: print(f'step{step} calculating...')
    
    predictions = np.concatenate(preds)

    return losses.avg, predictions


In [ ]:
#------------------------------------------------
# CV split
#------------------------------------------------

if CFG.debug:
    folds = train.sample(200).copy().reset_index(drop = True)
else:
    folds = train.copy()

folds['label'].value_counts()

kFold = StratifiedKFold(n_splits = 5, shuffle = True,
                        random_state = 11) 

for n ,(train_index, valid_index) in enumerate(kFold.split(folds, folds['label'])):
    folds.loc[valid_index, 'fold'] = n

folds.dtypes
folds['fold'] = folds['fold'].astype(int)
print(folds.groupby(['fold', 'label']).size())


In [ ]:
#------------------------------------------------
# trainig loop 
#------------------------------------------------

def train_loop(folds, fold):

    print(f'Fold: {fold + 1}')
                    
    aug_list = df_aug.query('\
                            item == "RRC" |\
                            item == "TR" |\
                            item == "HF" |\
                            item == "VF" |\
                            item == "SSR" |\
                            item == "NOR" |\
                            item == "TTR"')['augment'].values
    #
    
    train_folds = folds.loc[train_index].reset_index(drop = True)
    valid_folds = folds.loc[valid_index].reset_index(drop = True)
    
    train_dataset = SampleDataset(train_folds, 
                                transform = get_transforms(aug_list, data = "train"))
    
    train_loader = DataLoader(train_dataset, 
                               batch_size = 5,
                               #num_workers=2,
                               shuffle = True,
                               num_workers=CFG.num_workers, 
                               pin_memory=True, drop_last=True)
    
    
    valid_dataset = SampleDataset(valid_folds,
                                transform = get_transforms(aug_list, data = "valid"))
    valid_loader = DataLoader(valid_dataset,
                              batch_size = 10,
                              shuffle = False,
                              num_workers=CFG.num_workers, 
                               pin_memory=True, drop_last=False)
                               #ラベル順がわかるようにすうためシャフルしない
    
    model = CnnTrial()
    model.to(device)
    
    optimizer = Adam(model.parameters(), lr = 0.001)
    scheduler = ReduceLROnPlateau(optimizer = optimizer, verbose = True)
    criterion = nn.CrossEntropyLoss()
    
    best_score = 0.
    best_loss = np.inf
    best_model = []
    
    for epoch in range(CFG.epochs):
        
        print(f'Fold-{fold+1}: EPOCH {epoch+1} started at {datetime.datetime.now()}.')
        avg_loss = train_fn(train_loader, model, criterion, 
                            optimizer, epoch, scheduler, device)
        
        avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)
        
        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(avg_val_loss)
    
    
        valid_labels = valid_folds['label'].values
        
        preds_valid = preds.argmax(1)
        score = get_score(valid_labels, preds_valid)
        print(f'EPOCH {epoch+1}: vaildation score = {score:.3f}\n')
    
        if score > best_score: # FOLDの全EPOCH中のベストスコアを保存する
            best_score = score
            torch.save({'model': model.state_dict(),
                        'preds': preds},
                       os.path.join(OUTPUT_DIR , 'fold_'+str(fold+1)+'_best.pth'))
            best_model = model
        
       
    check_point = torch.load(os.path.join(OUTPUT_DIR , 'fold_'+str(fold+1)+'_best.pth'))
    #valid_folds[[str(c) for c in range(5)]] = check_point['preds'] 
    #自分のローカル環境では上記はエラーとなった。 
    
    for i in range(CFG.n_fold):
         valid_folds[f'pred_{i}'] = check_point['preds'][:, i]
    valid_folds['preds'] = check_point['preds'].argmax(1)
    
    return valid_folds, best_model

In [ ]:
#--------------------------------------------
# training
#--------------------------------------------
    
oof_df = pd.DataFrame()
best_models =[]
for fold in range(CFG.n_fold):
    _oof_df, best_model = train_loop(folds, fold)
    oof_df = pd.concat([oof_df, _oof_df])
    fold_score = get_result(_oof_df)
    print(f'fold {fold+1} score = {fold_score:.5f}')
    best_models.append(best_model)
    
oof_score = get_result(oof_df)
print(f'\nCV score = {oof_score:.5f}')

In [ ]:
#-----------------------------------
# inference
#-----------------------------------

class TestDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df
        self.file_names = df['image_id'].values
        #self.labels = df['label'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
        
    def __getitem__(self, index):
        file_name = self.file_names[index]
        #file_path = os.path.join(data_path, "train_images", file_name)
        file_path = os.path.join(data_path, "test_images", file_name)
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            augmented = self.transform(image = image)
            image_aug = augmented['image']
        
        image = image_aug
               
        #print("Return Shape = ", image.shape)
        
        return image
##

In [ ]:
test_dataset = TestDataset(test, transform=get_transforms(aug_list = [], data='valid'))
test_loader = DataLoader(test_dataset, batch_size=5, shuffle=False,
                        num_workers=CFG.num_workers, pin_memory=True)


from scipy.special import softmax

inferences = []
for i, (images) in enumerate(test_loader):
    #print(images)
    #print(images.shape)
    #print(images.size()[0])
    inference = np.zeros(images.shape[0]*5, dtype = 'float32').reshape(-1, 5)
    #print(inference.shape)
    images = images.to(device)
    #break

    for j in range(CFG.n_fold):
        model_inf = best_models[j]
        model_inf.to(device)
        model_inf.eval()
        with torch.no_grad():
            y_preds = model_inf(images)
            y_preds_np =  y_preds.cpu().numpy()
        inference += y_preds_np
    inference /= CFG.n_fold
    inference = softmax(inference, axis=1)
    inferences.append(np.argmax(inference, axis = 1))
   
test_inferences = np.concatenate(inferences, axis = 0)
test['label'] = test_inferences

test.to_csv(os.path.join(OUTPUT_DIR, "submission.csv"), index = False)